In [16]:
# load environment variables
!set -a
!source ../.env

In [27]:
from eval import get_dummy_doc
document = get_dummy_doc()


In [28]:
from app.chat.engine import fetch_and_read_document
document = fetch_and_read_document(document)     # merge document pages into a single document

LOADING DOCUMENT WITH SimpleDirectoryReader: META 10-K 2022
MERGING DOCUMENT: META 10-K 2022


In [33]:
print(f"len(document): {len(document)}")
print(f"type(document): {type(document)}")
print(f"type(document[0]): {type(document[0])}")

len(document): 1
type(document): <class 'list'>
type(document[0]): <class 'llama_index.schema.Document'>


In [35]:
import anyio
from app.chat.messaging import ChatCallbackHandler

send_chan, recv_chan = anyio.create_memory_object_stream(100)
callback_handler = ChatCallbackHandler(send_chan)

In [36]:
from app.chat.engine import get_tool_service_context
original_service_context = get_tool_service_context(callback_handlers=[callback_handler], node_parser_type="original")

In [69]:
# display original node parsing
original_node_parser = original_service_context.node_parser
original_nodes = original_node_parser.get_nodes_from_documents(document)

In [70]:
from eval import format_pdf_text
print(f"Total nodes: {len(original_nodes)}")
print(f"NODE {5} TEXT: {format_pdf_text(original_nodes[5].text)}")

Total nodes: 325
NODE 5 TEXT: We have based these forward-looking statements largely on our current expectations and projections about future events and trends that we believe may affect our financial condition, results of operations, business strategy, short-term and long-term business operations and objectives, and financial needs. These forward-looking statements are subject to a number of risks, uncertainties and assumptions, including those described in Part I, Item 1A, "Risk Factors" in this Annual Report on Form 10-K. Moreover, we operate in a very competitive and rapidly changing environment. New risks emerge from time to time. It is not possible for our management to predict all risks, nor can we assess the impact of all factors on our business or the extent to which any factor, or combination of factors, may cause actual results to differ materially from those contained in any forward-looking statements we may make. In light of these risks, uncertainties and assumptions, the 

In [83]:
sentence_window_service_context = get_tool_service_context(callback_handlers=[callback_handler], node_parser_type="setence-window")
sentence_window_node_parser = sentence_window_service_context.node_parser
sentence_window_nodes = sentence_window_node_parser.get_nodes_from_documents(document)

print(f"Total nodes: {len(sentence_window_nodes)}")
sentence = format_pdf_text(sentence_window_nodes[5].metadata.get("original_text"))
window = format_pdf_text(sentence_window_nodes[5].metadata.get("window"))

print(f"ORIGINAL SENTENCE: {sentence}")
print("------------------")
print(f"WINDOW: {window}")


Total nodes: 2184
ORIGINAL SENTENCE: Yes ☐ No ☒ Indicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 (Exchange Act) during the preceding 12 months (or for such shorter period that the registrant was required to file such reports), and (2) has been subject to such filing requirements for the past 90 days.
------------------
WINDOW: (Exact name of registrant as specified in its charter) __________________________ Delaware 20-1665019 (State or other jurisdiction of incorporation or organization) (I.R.S. Employer Identification Number) 1601 Willow Road , Menlo Park , California 94025 (Address of principal executive offices and Zip Code) ( 650 ) 543-4800 (Registrant's telephone number, including area code) __________________________ Securities registered pursuant to Section 12(b) of the Act: Title of each class Trading symbol(s) Name of each exchange on which registered Class A Common Stock